In [ ]:
import pandas as pd 
from BorutaShap import BorutaShap
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def BorutaShap_FS_ten (X, y,method_option) :
    #modelshap = RandomForestClassifier(n_jobs=-1,n_estimators=100, class_weight='balanced_subsample', max_depth=5, random_state=100)
    modelshap = RandomForestClassifier(n_jobs=-1,n_estimators=100, max_depth=5, random_state=100)

    # define model for resp. classifier
    modelshap.fit(X,y)
    feature_names = np.array(X.columns)
    # define Boruta Sahp feature selection method
    feature_selector = BorutaShap(model=modelshap,
                              importance_measure=method_option,
                              classification=True)  # find all relevant features
    feature_selector.fit(X,y,n_trials=100,sample = False, verbose = True,random_state=100)  
    #feature_selector.plot(which_features='accepted',figsize=(20,10))
    tentative=retransformed_train.loc[:,feature_selector_test.tentative]
    selected=feature_selector_test.Subset()
    selten=pd.concat([selected,tentative],axis=1)
    # call transform() on X to filter it down to selected features
    return  selten

In [ ]:
list_method_t=['shap','gini']
list_borshap_t = []
#order=0

for m in list_method_t:
    Output_t = BorutaShap_FS_ten(retransformed_train,y_train,m)
    list_borshap_t.append(Output_t)
    
for r in range(len(list_borshap_t)):
    print(list_borshap_t[r])

# Boruta Py with rfpimp and percentile threshold =80 for shadow comparison

In [ ]:
def Boruta_FS_MDA (X, y) :
    feature_names_rfpimp = np.array(X.columns)

    # define random forest classifier
    model_rfpimp = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5,oob_score=True, random_state=100)
    model_rfpimp.fit(X, y)
    # define Boruta feature selection method
    
    feature_selector_rfpimp = BorutaPy(model_rfpimp, n_estimators='auto', verbose=2, random_state=100, max_iter=140, perc=80)

    # find all relevant features
    feature_selector_rfpimp.fit(X.to_numpy(),y)

    # check selected features
    feature_selector_rfpimp.support_

    # check ranking of features
    feature_ranking_rfpimp=feature_selector_rfpimp.ranking_

    # zip feature names, ranks, and decisions 
    feature_ranks_rfpimp = list(zip(feature_names_rfpimp, 
                             feature_selector_rfpimp.ranking_, 
                             feature_selector_rfpimp.support_))

    # print the results
    for feat in feature_ranks_rfpimp:
        print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features_rfpimp = list()
    indexes_rfpimp = np.where(feature_selector_rfpimp.ranking_ <= 1)
    for x in np.nditer(indexes_rfpimp):
        final_features_rfpimp.append(feature_names_rfpimp[x])
    print(final_features_rfpimp)
    
 # call transform() on X to filter it down to selected features
    return pd.DataFrame(X.filter(final_features_rfpimp)) , final_features_rfpimp, feature_ranking_rfpimp

In [ ]:
X_train_mds , final_features_mds, feature_ranking_mda = Boruta_FS_MDA(retransformed_train,y_train)# 140 iteration , per=80 , rank=1

In [ ]:
X_train_mds_dup=remove_duplicated_columns(X_train_mds)

In [ ]:
X_train_mds_cor=remove_collinear_features(X_train_mds,0.2)

### RFE function to return unscaled features 

In [ ]:
#RFE function with random forest

def RFE_FS (X, y,classify) :
    feature_names = np.array(X.columns)
    if classify == 'RF':
    # define random forest classifier
        model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
        #model.fit(X, y)
        #rfe = RFE(estimator = model,n_features_to_select = 15)
    if classify== 'SVM':
        model = SVC(kernel='linear',C=5)
        #model.fit(X, y)
        #rfe = RFECV(estimator = model,scoring='accuracy')
    # find all relevant features
    model.fit(X, y)
    rfe = RFE(estimator = model,n_features_to_select = 15)
    rfe.fit(X,y)

    # check selected features
    rfe.support_

    # check ranking of features
    rfe.ranking_

    # zip feature names, ranks, and decisions 
    feature_ranks = list(zip(feature_names, 
                             rfe.ranking_, 
                             rfe.support_))

    # print the results
    for feat in feature_ranks:
        print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features_rfe = list()
    indexes = np.where(rfe.ranking_ <= 1)
    for x in np.nditer(indexes):
        final_features_rfe.append(feature_names[x])
    print(final_features_rfe)
    
    # unscale the data before return
    ff_rfe_scaled=pd.DataFrame(X.filter(final_features_rfe))
    ff_rfe_unscaled=pd.DataFrame(scaler.inverse_transform(X), columns=X.columns)

 # call transform() on X to filter it down to selected features
    return  ff_rfe_unscaled, final_features_rfe